# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 7.21MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
  Created wheel for Chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7095 sha256=5aae92f50f72ecf16c8ce34f8b7e5652a8459490da308e1258f09bda83289308
  Stored in directory: /root/.cache/pip/wheels/58/74/75/a6ab7999ae473ecbe819bc5cae9ccb902429dd6c60795f5112
Successfully built Chroma


In [4]:
!pip install langchain

In [5]:
!pip install langchain-groq

In [6]:
from langchain.chains import LLMChain
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import pandas as pd
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [7]:
df = pd.read_csv("Dataset.csv")
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [8]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [9]:
df.duplicated().sum()

0

In [10]:
df.isna().sum()

,0
المخالفة,0
الغرامة,0


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [11]:
#the markdown files will be stored
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [12]:
#empty list to store the markdown text
markdown_list = []


for index, row in df.iterrows():

    violation = row['المخالفة']
    fine = row['الغرامة']
    markdown = f"المخالفة: {violation} - الغرامة: {fine}"

    markdown_list.append(markdown)
    with open(f'{directory}/{index}.md', 'w', encoding='utf-8') as file:
        file.write(markdown)


In [15]:
markdown_list[5:10]

['المخالفة: وقوف المركبة في أماكن غير مخصصة للوقوف. - الغرامة: الغرامة المالية 100 - 150 ريال',
 'المخالفة: عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم. - الغرامة: الغرامة المالية 100 - 150 ريال',
 'المخالفة: عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً. - الغرامة: الغرامة المالية 150 - 300 ريال',
 'المخالفة: قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر. - الغرامة: الغرامة المالية 150 - 300 ريال']

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [16]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.create_documents(markdown_list)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [17]:
embeddings_model = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/Arabic-Nli-Matryoshka")
db = Chroma.from_documents(chunks, embeddings_model, persist_directory="./chroma_db")

<ipython-input-17-847f30c5468c>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings_model = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/Arabic-Nli-Matryoshka")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a to

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/35.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:

#list the most similar documents
def search_chroma_database(query_text, database, num_results=5):

    similar_docs = database.similarity_search(query_text, k=num_results)
# Extract
    search_results = [document.page_content for document in similar_docs]

    return search_results


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [19]:
# Definethe {query} represents the user's question
PROMPT_TEMPLATE = """
السؤال: {query}
الغرامة: {context}
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "query"]
)


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [21]:
# from langchain.llms import Groq
from langchain_groq import ChatGroq

In [22]:
# Set the API key
groq_api_key = "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"


# The model selected is llama3-8b-8192
llm = ChatGroq(
    temperature=0.7,
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192"
)


## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [23]:
# The 'verbose=1' parameter enables detailed logging,
rag_chain = LLMChain(llm=llm,
                     prompt=prompt_template,
                     verbose=1)

<ipython-input-23-856a710e1a24>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  rag_chain = LLMChain(llm=llm,


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [24]:
def query_rag(query: str):
    #retrieving the top 4 results.
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    # Invok the RAG chain
    rag_response = rag_chain.invoke({"context": context_text, "query": query})
    return rag_response


## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [25]:

user_query = "ما هي غرامة الانشغال بغير الطريق"
response = query_rag(user_query)
response


Prompt after formatting:

السؤال: ما هي غرامة الانشغال بغير الطريق
الغرامة: المخالفة: الانشغال بغير الطريق أثناء قيادة المركبة. - الغرامة: الغرامة المالية 300 - 500 ريال

المخالفة: عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز. - الغرامة: الغرامة المالية 500 - 900 ريال

المخالفة: استعمال المركبة لغير الغرض الذي رخصت من أجله. - الغرامة: الغرامة المالية 500 - 900 ريال

المخالفة: عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك. - الغرامة: الغرامة المالية 500 - 900 ريال


> Finished chain.


{'context': 'المخالفة: الانشغال بغير الطريق أثناء قيادة المركبة. - الغرامة: الغرامة المالية 300 - 500 ريال\n\nالمخالفة: عدم قيام السائق الذي يرغب في تغيير مساره بإعطاء الأفضلية لسائق المركبة التي تسير في اتجاه مستقيم في حال سير المركبتين متحاذيتين بشكل متواز. - الغرامة: الغرامة المالية 500 - 900 ريال\n\nالمخالفة: استعمال المركبة لغير الغرض الذي رخصت من أجله. - الغرامة: الغرامة المالية 500 - 900 ريال\n\nالمخالفة: عدم إعطاء أفضلية المرور في ملتقيات الطرق أو تقاطعاتها لسائق المركبة المتقدم على غيره في حال عدم وجود لوحات تنظم ذلك. - الغرامة: الغرامة المالية 500 - 900 ريال',
 'query': 'ما هي غرامة الانشغال بغير الطريق',
 'text': 'According to the text, the fine for being distracted from the road during driving is:\n\n300 - 500 Riyals'}